In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset

from models.networks import get_model
from data_utils.data_stats import *


dataset = 'cifar10'                 # One of cifar10, cifar100, stl10, imagenet or imagenet21
architecture = 'B_12-Wi_1024'
data_resolution = 32                # Resolution of data as it is stored
crop_resolution = 64                # Resolution of fine-tuned model (64 for all models we provide)
num_classes = CLASS_DICT[dataset]
data_path = './beton/'
eval_batch_size = 1024
checkpoint = 'in21k_cifar10'        # This means you want the network pre-trained on ImageNet21k and finetuned on CIFAR10



# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- Load the CIFAR-10 test dataset -------------

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(64),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2471, 0.2435, 0.2616)),
])

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

#--------- Load the permuted dataset ----------

class CustomDataset(Dataset):
    def __init__(self, dataset_dict, transform = None):
        self.dataset_dict = dataset_dict
        self.transform = transform

    def __len__(self):
        return len(self.dataset_dict)

    def __getitem__(self, index):
        data = self.dataset_dict[index]
        image = data['image']
        if self.transform:
            image = self.transform(data['image'])
        label = data['label']

        return image, label

# Load the dataset dictionary
loaded_dataset = torch.load('cifar10_permuted.pth')

# Create an instance of the CustomDataset
permuted_dataset = CustomDataset(loaded_dataset, transform = transform)
permuted_loader = DataLoader(permuted_dataset, batch_size=1024, shuffle=False)


# ------------- Load the pretrained model ------------

pretrained_model = get_model(architecture=architecture, resolution=crop_resolution, num_classes=CLASS_DICT[dataset],
                  checkpoint='in21k_cifar10')
pretrained_model = pretrained_model.to(device)
pretrained_model.eval()

# -----------calculate accuracies -----------
correct_predictions = 0
total_samples = 0

correct_predictions_permuted = 0
total_samples_permuted = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        
        inputs = torch.reshape(inputs, (inputs.shape[0], -1))
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = pretrained_model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Update counts
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

    for inputs, labels in permuted_loader:
        
        inputs = torch.reshape(inputs, (inputs.shape[0], -1))
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = pretrained_model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Update counts
        total_samples_permuted += labels.size(0)
        correct_predictions_permuted += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f'Accuracy on CIFAR-10: {100 * accuracy:.2f}%')

# Calculate accuracy on permuted dataset
accuracy_permuted = correct_predictions_permuted / total_samples_permuted
print(f'Accuracy on permuted CIFAR-10: {100 * accuracy_permuted:.2f}%')

/Users/charleslego/opt/anaconda3/envs/DL_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Files already downloaded and verified
Weights already downloaded
Load_state output <All keys matched successfully>


/Users/charleslego/opt/anaconda3/envs/DL_env/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Accuracy on CIFAR-10: 94.09%
Accuracy on permuted CIFAR-10: 31.30%


In [ ]:
# --------- Code for per class accuracy -------------

# Initialize a dictionary to store per-class counts
class_counts = {class_idx: {'correct': 0, 'total': 0} for class_idx in range(num_classes)}

with torch.no_grad():
    for inputs, labels in test_loader:

        inputs = torch.reshape(inputs, (inputs.shape[0], -1))
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = pretrained_model(inputs)
        _, predicted = torch.max(outputs, 1)

        # Update counts for each class
        for class_idx in range(num_classes):
            class_mask = labels == class_idx
            class_total = class_mask.sum().item()
            class_correct = (predicted[class_mask] == class_idx).sum().item()

            class_counts[class_idx]['correct'] += class_correct
            class_counts[class_idx]['total'] += class_total

# Calculate per-class accuracy
per_class_accuracy = {class_idx: class_counts[class_idx]['correct'] / class_counts[class_idx]['total']
                      for class_idx in range(num_classes)}


# Print per-class accuracy
for class_idx in range(num_classes):
    print(f'Accuracy for class {class_idx}: {100 * per_class_accuracy[class_idx]:.2f}%')


#0: airplanes, 1: cars, 2: birds, 3: cats, 4: deer, 5: dogs, 6: frogs, 7: horses, 8: ships, 9: trucks